In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
from sklearn.linear_model import LogisticRegression
import mglearn 
from sklearn.preprocessing import MinMaxScaler

fashion_df = pd.read_csv("./data/fashionmnist/fashion-mnist_train.csv")
fashion_df.head()



,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# train, test 데이터 생성
# 정규화
scaler= MinMaxScaler()
train_num = int(fashion_df.shape[0]*0.8) 

x_data = fashion_df.drop("label",axis=1,inplace=False)
x_data = scaler.fit_transform(x_data)

y_data = sess.run(tf.one_hot(fashion_df["label"], 10))

# 데이터 분할해 x data 생성 - 80:20
train_x_data = x_data[:train_num]  # nparray에서 자른 값 ->  뒤의 값이 exclusive 해서 16000개
test_x_data = x_data[train_num:]

# 학습용, 테스트용 y data 생성 -> one hot encoding
sess = tf.Session()
train_y_data = sess.run(tf.one_hot(fashion_df.loc[:train_num-1,"label"], 10)) # 원래 1차원 자료가 one-hot encoding 적용하면 2차원 자료가 된다.
# sess.run(tf.one_hot(bmi_df.loc[:train_num-1,"label"], 3)).shape # data frame이라 loc 사용하면 뒤부분 inclusive하다 -> train_num -1 빼줘야 함! 
test_y_data = sess.run(tf.one_hot(fashion_df.loc[train_num:,"label"], 10))

In [18]:
# CNN
# 그래프를 초기화해보자! 
tf.reset_default_graph() 

# placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
keep_rate = tf.placeholder(dtype=tf.float32) # drop out 비율

# Convolution Layer
x_img = tf.reshape(X,[-1,28,28,1])  # -1: 몇장인지 모름    # convolution하려면 무조건 4차원이어야 함!
F1 = tf.Variable(tf.random_normal(shape=[3,3,1,32]))  #  32개의 filter map
    # 여기서는 자비에르초기법 하는거 아님. W의 초기값을 랜덤으로 받는다는 건데 그건 머신러닝때. 지금은 filter를 랜덤으로 잡는 과정이므로 랜덤해야한다.
L1 = tf.nn.conv2d(x_img, F1, strides=[1,1,1,1], padding="SAME")
L1 = tf.nn.relu(L1)  # convolution 하고 relu 해줘야 함(값 너무 커지지 않게)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
# -> 14*14 크기가 32개 나옴

F2 = tf.Variable(tf.random_normal(shape=[3,3,32,64]))  # 32: 앞에서 나오는 channel 수
    # 3*3 크기의 32 depth를 64번 filter하겠다
L2 = tf.nn.conv2d(L1, F2, strides=[1,1,1,1], padding="SAME")
L2 = tf.nn.relu(L2)  
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")    
# -> 7*7 크기가 64개 나옴  


# 이렇게 만든 데이터를 FC Layer에 넣어서 학습해야 한다.
L2 = tf.reshape(L2,[-1,7*7*64])
W1 = tf.get_variable("weight1", shape=[7*7*64,256],
                    initializer=tf.contrib.layers.xavier_initializer())

########################################################
b1 = tf.Variable(tf.random_normal([256]),name="bias1")
_layer1 = tf.nn.relu(tf.matmul(L2,W1)+b1)
layer1 = tf.nn.dropout(_layer1, keep_prob=keep_rate) # 256개  output을 다 뽑아내지 않겠다. node를 아예 삭제하는게 아니라 기능을 상실시키는 것
                                         # rate=0 하면 다 살아있는거   0.3하면 30% 죽이는거 -> 다음으로 30프로 죽여서 넘기겠다.
     # drop_rate가 cpu_env에서는 keep_rate
W2 = tf.get_variable("weight2", 
                     shape=[256,256], 
                     initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]),name="bias2")
_layer2 = tf.nn.relu(tf.matmul(layer1,W2)+b2)
layer2 = tf.nn.dropout(_layer2,keep_prob=keep_rate) 

W3 = tf.get_variable("weight3", 
                     shape=[256,10], 
                     initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]),name="bias3")  # 맨 마지막에는 dropout해주지 않는다. -> 다 넘겨야 하므로
logit = tf.matmul(layer2,W3)+b3
H = tf.nn.relu(logit)  

# cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,labels=Y))  

# train
#train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.005).minimize(cost)  # 이거도 자주쓰임. 크게 성능의 향상 보기엔 어렵지만 좀더 성능 좋다는게 일반적인 생각

# session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
num_of_epoch =30  # 반복횟수
batch_size = 100

for step in range(num_of_epoch): 
    num_of_iter = int(train_num/batch_size)
    cost_val = 0
    
    for i in range(num_of_iter):
        batch_x = x_data[batch_size*i:batch_size*(i+1)]
        batch_y = y_data[batch_size*i:batch_size*(i+1)]        
        _, cost_val = sess.run([train,cost],feed_dict = {X:batch_x,
                                                         Y:batch_y,
                                                         keep_rate:0.7}) # 30% 끄고 학습해라
    if step%3==0:
        print("cost:{}".format(cost_val))

cost:0.7815183997154236
cost:0.4112594723701477
cost:0.36653557419776917
cost:0.25425422191619873
cost:0.216287761926651
cost:0.27551594376564026
cost:0.3250957429409027
cost:0.20296451449394226
cost:0.15294292569160461
cost:0.1376994103193283


In [9]:
# accuracy 측정
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

print("정확도:{}".format(sess.run(accuracy, feed_dict={X:test_x_data,
                                                       Y:test_y_data,
                                                       keep_rate:1})))

정확도:0.8800833225250244


In [23]:
# 진짜 test 파일로 돌려보자!
test_df = pd.read_csv("./data/fashionmnist/fashion-mnist_test.csv")
test_df = test_df.drop("label",axis=1,inplace=False)
# MinMax scaler가 min, max값 가지고 있다.
#prediction_df = scaler.transform(test_data)

#sess.run(H,feed_dict={X:prediction_data})
result = sess.run(H, feed_dict={X:test_df})

predict = tf.cast(result > 0.5,dtype=tf.int32)
result = sess.run(predict)


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_2', defined at:
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\torn

In [ ]:
my_df = pd.DataFrame()
my_df["ImageId"] = range(1,test_data.shape[0]+1)
my_df["Label"] = result
my_df

In [ ]:
result.to_csv('./data/titanic/submission_deep_dropout.csv',
                 sep=',',
                 na_rep='NaN', 
                 columns = ['PassengerId','Survived'], # columns to write
                 index = False) # do not write index
